#  APP INITIALIZATION

In [4]:
from tkinter.messagebox import RETRY
from xml.dom.expatbuilder import DOCUMENT_NODE

#LIBRERIAS
import firebase_admin
from firebase_admin import auth #Authentication service
from firebase_admin import storage #Storage service for User Profile pictures
from firebase_admin import credentials
from firebase_admin import firestore #NoSQL database

# Use a service account.
cred = credentials.Certificate('/home/aldo/PycharmProjects/scientificProject/ingebot-5c0de-firebase-adminsdk-46g76-1519602298.json')
#App initialization with credentials for the project nad access to the bucket (Storage for images)
app = firebase_admin.initialize_app(cred,{
    'storageBucket': 'ingebot-5c0de.appspot.com' #Place where all images are located
})

db = firestore.client()

## BUCKET REFERENCE 
This is where all profile pictures will be uploaded

In [2]:
firebase_admin.delete_app(app)

In [5]:
bucket = storage.bucket(app=app)

### CREATING NEW USERS
The create_user function allows you to create a new user and upload its info into the following services
- Authentication 
- Firestore 
- Storage (if image is uploaded)



This function returns a dictionary with the following structure:
        
    user_data = {
            'name': display_name,
            'institution_email': institution_email,
            'createdAt': firestore.SERVER_TIMESTAMP,  # Use Firestore's server timestamp
            'control_number': control_number,
            'photoUrl': image_url,  # Profile image URL
            'uid': user.uid  # Store the UID for future reference
        }   
Note: If no image is provided, then the "photoUrl" attribute will be set to None




In [54]:


def create_user(institution_email, password, display_name, control_number,image_path=None ):


    try:
        # Step 1: Create the user in Firebase Authentication
        # Firebase Authentication only needs email and password, the other parameters are optional
        user = auth.create_user(
            email=institution_email,
            email_verified=False,
            password=password,
            display_name=display_name,
            disabled=False
        )
        print(f'Successfully created new user: {user.uid}')
        
        # Step 2: Upload profile picture if provided
        if image_path:
            image_url = upload_image_and_get_url(image_path, user.uid)
            print(f'Profile image URL: {image_url}')
        else:
            image_url = None
        
        # Step 3: Store user data in Firestore
        user_data = {
            'name': display_name,
            'institution_email': institution_email,
            'createdAt': getCurrentTimestamp(),  # Use Firestore's server timestamp
            'control_number': control_number,
            'photoUrl': image_url,  # Profile image URL
            'uid': user.uid  # Store the UID for future reference
        }

        # Step 4: Add the user document to Firestore
        user_ref = db.collection('Users').document(user.uid)
        user_ref.set(user_data)
        return user
        
    except Exception as e:
        print(f'Error creating user or adding to Firestore: {str(e)}')

#### Upload image function
The function upload_image_and_get_url allows you to upload images from your device, you only need to specify the user uid since the image will only be identified with this attribute

In [13]:
def upload_image_and_get_url(image_path, user_uid):
    """Upload a profile image to Firebase Storage and return its URL."""
    try:
        
        # Create a blob (object) for the image in Storage
        blob = bucket.blob(f'user_profile_pictures/{user_uid}/{image_path.split("/")[-1]}')  # Storage path
        blob.upload_from_filename(image_path)  # Upload the image file

        # Make the blob publicly accessible (optional)
        blob.make_public()

        # Return the URL of the uploaded image
        return blob.public_url
    except Exception as e:
        print(f'Error uploading image: {str(e)}')
        return None


#### getCurrentTimestamp function
This function allows you to obtain current a formatted timestamp 

In [14]:

def getCurrentTimestamp():
    from datetime import datetime
    current_timestamp = datetime.now()
    formatted_timestamp = current_timestamp.strftime('%Y-%m-%d %H:%M:%S')
    return formatted_timestamp
    
    

USER CREATION EXAMPLE


In [78]:
user4 =create_user(
    institution_email='7@example.com',
    password='<PASSWORD>',
    display_name='4',
    control_number='22090747',
    image_path= '/home/aldo/PycharmProjects/scientificProject/aldo.jpg'
)

Successfully created new user: GGh07R3wtdets9UYiHmRTxuGHRJ3
Profile image URL: https://storage.googleapis.com/ingebot-5c0de.appspot.com/user_profile_pictures/GGh07R3wtdets9UYiHmRTxuGHRJ3/aldo.jpg


RETRIEVE USER DATA

In [24]:
userEmail = user4['institution_email']
userUid = user4['photoUrl']
userName = user4['name']
print(f'User Name: {userName} \n User UID: {userUid} \n User Email: {userEmail}')

User Name: 4 
 User UID: https://storage.googleapis.com/ingebot-5c0de.appspot.com/user_profile_pictures/8KMGTr1MfmdT13CLUQs3Gj2rvRE3/aldo.jpg 
 User Email: 6@example.com


## Creating Tasks

In [80]:
def createTask(title, description, due_date, priority):
    userUID= user4.uid
    taskData={
        'title': title,
        'description': description,
        'createdAt': getCurrentTimestamp(),  # Use Firestore's server timestamp
        'due_date': due_date, #EXAMPLE ONLY , WE NEED TO CHANGE IT  
        'priority':priority,
    }
    user_ref = db.collection('Users').document(userUID)
   
    new_task_ref = user_ref.collection('Tasks').document(title)  

    # Set the task data in Firestore
    new_task_ref.set(taskData)
    

    
 
 
 

In [162]:
createTask(
    title='almorzar',
    description='This is a test task',
    due_date="2024-01-19",
    priority='HIGH'
)

### Creating subtasks

In [104]:
def createSubtask(taskTitle, subtaskTitle, description):
    userUID= user4.uid 
    subaskData={
        'subtaskTitle': subtaskTitle,
        'description': description,
        'createdAt': getCurrentTimestamp(),  # Use Firestore's server timestamp
    }
    task_ref = db.collection('Users').document(userUID).collection('Tasks')
    task_ref.document(taskTitle).collection('Subtasks').document(subtaskTitle).set(subaskData)
    

In [164]:
createSubtask(
    taskTitle='almorzar',
    subtaskTitle='Ir al baño',
    description='This is a test subtask',
)

## READING DATA

### User data
The following sentences retrieves the user's information in dictionary, not that it does not retrieve the user's subcollections ("Tasks")

- User document without subcollection "Tasks"


In [175]:
db.collection("Users").document(user4.uid).get().to_dict()


{'name': '4',
 'uid': 'GGh07R3wtdets9UYiHmRTxuGHRJ3',
 'photoUrl': 'https://storage.googleapis.com/ingebot-5c0de.appspot.com/user_profile_pictures/GGh07R3wtdets9UYiHmRTxuGHRJ3/aldo.jpg',
 'createdAt': '2024-10-20 18:09:27',
 'institution_email': '7@example.com',
 'control_number': '22090747'}

### User Tasks

- Subcollections of a user

In [149]:
subcollections = db.collection("Users").document(user4.uid).collections()
for subcollection in subcollections:
    print(f'Subcollection: {subcollection.id}')

Subcollection: Tasks


In [88]:
db.collection("Users").document(user4.uid).collection("Tasks").document("desayunar").get().to_dict()

{'title': 'desayunar',
 'due_date': '2024-01-19',
 'description': 'This is a test task',
 'createdAt': '2024-10-20 18:10:03',
 'priority': 'HIGH'}

- Subcollections of a task

In [150]:
subcollectionst = db.collection("Users").document(user4.uid).collection("Tasks").document("desayunar").collections()
for subcollection in subcollectionst:
    print(f'Subcollection: {subcollection.id}')


Subcollection: Subtasks


### Collection Users

In [91]:
docs = db.collection("Users").stream()
for doc in docs:
    print(f'Document ID: {doc.id}, Data: {doc.to_dict()}')

Document ID: GGh07R3wtdets9UYiHmRTxuGHRJ3, Data: {'name': '4', 'uid': 'GGh07R3wtdets9UYiHmRTxuGHRJ3', 'photoUrl': 'https://storage.googleapis.com/ingebot-5c0de.appspot.com/user_profile_pictures/GGh07R3wtdets9UYiHmRTxuGHRJ3/aldo.jpg', 'createdAt': '2024-10-20 18:09:27', 'institution_email': '7@example.com', 'control_number': '22090747'}


### Collection Tasks

In [172]:
docs = db.collection("Users").document(user4.uid).collection("Tasks").stream()
for doc in docs:
    print(f'Document ID: {doc.id}, Data: {doc.to_dict()}')

Document ID: almorzar, Data: {'title': 'almorzar', 'due_date': '2024-01-19', 'description': 'This is a test task', 'createdAt': '2024-10-20 19:08:08', 'priority': 'HIGH'}
Document ID: desayunar, Data: {'title': 'desayunar', 'due_date': '2024-01-19', 'description': 'This is a test task', 'createdAt': '2024-10-20 18:20:20', 'priority': 'HIGH'}


### Collections Subtasks

In [138]:
subtasks1 = db.collection("Users").document(user4.uid).collection("Tasks").document("desayunar").collection("Subtasks").stream()
for subtask in subtasks1:
    print(f'Document ID: {subtask.id}, Data: {subtask.to_dict()}')

Document ID: Abrir refri, Data: {'description': 'This is a test subtask', 'createdAt': '2024-10-20 18:21:52', 'subtaskTitle': 'Abrir refri'}
Document ID: Comer, Data: {'description': 'This is a test subtask', 'createdAt': '2024-10-20 18:22:04', 'subtaskTitle': 'Comer'}
Document ID: Tomar plato, Data: {'description': 'This is a test subtask', 'createdAt': '2024-10-20 18:21:34', 'subtaskTitle': 'Tomar plato'}


# Read a user document in a json format

In [173]:

import json


def fetch_subcollections(document_ref):
    """
    Recursively fetch subcollections for a given Firestore document reference.
    """
    data = {}
    
    # Get the document data
    document_data = document_ref.get().to_dict()
    
    # Add the document data to the result
    if document_data:
        data['data'] = document_data
    
    # Get subcollections
    subcollections = document_ref.collections()
    
    for subcollection in subcollections:
        # Recursively call this function for each subcollection
        subcollection_data = []
        for doc in subcollection.stream():
            subcollection_data.append(fetch_subcollections(doc.reference))
        data[subcollection.id] = subcollection_data
    
    return data

def print_user_info(user_id):
    """
    Fetch all user information from Firestore, including subcollections, and print it to the screen.
    """
    # Reference to the user's document
    user_ref = db.collection("Users").document(user_id)
    
    # Fetch user document and its subcollections
    user_data = fetch_subcollections(user_ref)
    
    # Print the result as a formatted JSON string
    print(json.dumps(user_data, indent=4))
    
# Example usage:
user_id = user4.uid # The user ID of the user you want to print

# Print the user information to the screen
print_user_info(user_id)



{
    "data": {
        "name": "4",
        "uid": "GGh07R3wtdets9UYiHmRTxuGHRJ3",
        "photoUrl": "https://storage.googleapis.com/ingebot-5c0de.appspot.com/user_profile_pictures/GGh07R3wtdets9UYiHmRTxuGHRJ3/aldo.jpg",
        "createdAt": "2024-10-20 18:09:27",
        "institution_email": "7@example.com",
        "control_number": "22090747"
    },
    "Tasks": [
        {
            "data": {
                "title": "almorzar",
                "due_date": "2024-01-19",
                "description": "This is a test task",
                "createdAt": "2024-10-20 19:08:08",
                "priority": "HIGH"
            },
            "Subtasks": [
                {
                    "data": {
                        "description": "This is a test subtask",
                        "createdAt": "2024-10-20 19:10:09",
                        "subtaskTitle": "Ir al ba\u00f1o"
                    }
                },
                {
                    "data": {
              

# Read subtasks for a particular task in a json format

In [171]:

import json


def fetch_subtasks(user_id, task_name):
    """
    Fetch all subtasks for a given user and task, and return the data as a dictionary.
    """
    subtasks_data = []

    # Reference to the subtasks collection
    subtasks_ref = db.collection("Users").document(user_id).collection("Tasks").document(task_name).collection("Subtasks")

    # Stream through all subtasks
    subtasks1 = subtasks_ref.stream()
    
    for subtask in subtasks1:
        # Append each subtask's data to the list
        subtasks_data.append({
           
            "data": subtask.to_dict()
        })
    
    return subtasks_data

def print_subtasks_in_json(user_id, task_name):
    """
    Fetch all subtasks for a user and task, and print them in JSON format.
    """
    # Fetch the subtasks
    subtasks = fetch_subtasks(user_id, task_name)
    
    # Print the subtasks in a formatted JSON string
    print(f'Subtasks for task "{task_name}" under the Uid: {user_id}')
    print(json.dumps(subtasks, indent=4))

# Example usage:
user_id = user4.uid  # The user ID of the user whose subtasks you want to fetch
task_name = 'desayunar'  # The task name for which you want to fetch subtasks

# Print the subtasks in JSON format
print_subtasks_in_json(user_id, task_name)



Subtasks for task "desayunar" under the Uid: GGh07R3wtdets9UYiHmRTxuGHRJ3
[
    {
        "data": {
            "description": "This is a test subtask",
            "createdAt": "2024-10-20 18:21:52",
            "subtaskTitle": "Abrir refri"
        }
    },
    {
        "data": {
            "description": "This is a test subtask",
            "createdAt": "2024-10-20 18:22:04",
            "subtaskTitle": "Comer"
        }
    },
    {
        "data": {
            "description": "This is a test subtask",
            "createdAt": "2024-10-20 18:21:34",
            "subtaskTitle": "Tomar plato"
        }
    }
]


## Deleting user's information
- DELETE A SUBTASK
